# SageMaker JumpStart Foundation Models - BloomZ 176B FP16 Few-shot and Zero-shot learning

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

---

---
Welcome to Amazon [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html)! You can use Sagemaker JumpStart to solve many Machine Learning tasks through one-click in SageMaker Studio, or through [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#use-prebuilt-models-with-sagemaker-jumpstart).


In this demo notebook, we demonstrate how to use the SageMaker Python SDK for deploying Foundation Models as an endpoint and use them for various NLP tasks. The Foundation models perform **Text Generation**. It takes a text string as input and predicts a sequence of next few words. These models can, for example, fill in incomplete text or paraphrase.

Here, we show how to use the state-of-the-art instruction-tuned **BloomZ 176B model** from [Hugging Face](https://huggingface.co/bigscience/bloomz) for **Text Generation** in the following tasks. You can use **BloomZ 176B model** model with **few shot learning** and **zero shot learning** for many NLP tasks, without fine-tuning the model. 

The **few shot leanring** allows a machine learning model to make predictions with only a couple of examples. There is no need to train a new model as models like **BloomZ 176B model** have a significant amount of parameters such that they can easily adapt to many contexts without being re-trained. Below are tasks that text generation can solve using few shot learning method.

* **Few-shot learning**
    * **Text summarization**
    * **Code generation**
    * **Name entity recognition (NER)**
    * **Question answering**
    * **Grammar and spelling correction**
    * **Product description and generalization**
    * **Sentence / sentiment classification**
    * **Chatbot and conversational AI**
    * **Tweet generation**
    * **Machine translation**
    * **Intent classification**
* **Zero-shot learning**
    * **Multilingual text / sentiment classification**
    * **Multilingual question and answering**
    * **Code generation**
    * **Paragraph rephrase**
    * **Summarization**
    * **Common sense reasoning / natural language inference**
    * **Question and answering**
    * **Sentence / sentiment classification**
    * **Imaginary article generation based on title**
    * **Summarize a title based on a article**
    
<font color='red'>**Note. Deploying this model requires a `p4de.24xlarge` instance and the deployment usually takes about 1 hour.**</font> 

**If you don't have quota for such instance, please go to AWS console -> Service Quotas -> AWS services to request it.**

---

1. [Set Up](#1.-Set-Up)
2. [Select a model](#2.-Select-a-model)
3. [Retrieve Artifacts & Deploy an Endpoint](#3.-Retrieve-Artifacts-&-Deploy-an-Endpoint)
4. [Query endpoint and parse response](#4.-Query-endpoint-and-parse-response)
5. [Advanced features: How to use varisous parameters to control the generated text](#5.-Advanced-features:-How-to-use-various-advanced-parameters-to-control-the-generated-text)
6. [Advanced features: How to use prompts engineering to solve different tasks](#6.-Advacned-features:-How-to-use-prompts-engineering-to-solve-different-tasks)
5. [Clean up the endpoint](#5.-Clean-up-the-endpoint)

Note: This notebook was tested on ml.t3.medium instance in Amazon SageMaker Studio with Python 3 (Data Science) kernel and in Amazon SageMaker Notebook instance with conda_python3 kernel.

### 1. Set Up

---
Before executing the notebook, there are some initial steps required for set up. This notebook requires ipywidgets.

---

In [ ]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker --quiet

#### Permissions and environment variables

---
To host on Amazon SageMaker, we need to set up and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook as the AWS account role with SageMaker access. 

---

In [5]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. Select a pre-trained model
***
You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).
***

In [11]:
model_id, model_version, = (
    "huggingface-textgeneration1-bloomz-176b-fp16",
    "1.*",
)

***
[Optional] Select a different Sagemaker pre-trained model. Here, we download the model_manifest file from the Built-In Algorithms s3 bucket, filter-out all the Text Generation models and select a model for inference.

The notebook in the following sections uses [**BloomZ 176B**](https://huggingface.co/bigscience/bloomz) as an example. You are welcome to try [**BloomZ 7b1**](https://huggingface.co/bigscience/bloomz-7b1), [**BloomZ 3b**](https://huggingface.co/bigscience/bloomz-3b), and many others yourself. Please modify `model_id` based on the dropdown list shown as below.
***

In [12]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# Retrieves all Text Generation models available by SageMaker Built-In Algorithms.
filter_value = "task == textgeneration1"
text_generation_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=text_generation_models,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

#### Choose a model for Inference

In [13]:
display(model_dropdown)

A Jupyter Widget

In [14]:
# model_version="*" fetches the latest version of the model
model_id, model_version = model_dropdown.value, "1.*"

### 3. Retrieve Artifacts & Deploy an Endpoint

***

Using SageMaker, we can perform inference on the pre-trained model, even without fine-tuning it first on a new dataset. We start by retrieving the `deploy_image_uri`, `deploy_source_uri`, and `model_uri` for the pre-trained model. To host the pre-trained model, we create an instance of [`sagemaker.model.Model`](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) and deploy it. This may take a few minutes.

***

In [8]:
model_id

'huggingface-textgeneration1-bloomz-176b-fp16'

**Now we deploy the model using simiplified JumpStart SDK with following few lines of code.**

<font color='red'>**Note. Deploying this model requires a `p4de.24xlarge` instance and the deployment usually takes about 1 hour.**</font>
**If you don't have quota for such instance, please go to AWS console -> Service Quotas -> AWS services to request it.**

In [18]:
import time
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.utils import name_from_base


endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

inference_instance_type = "ml.p4de.24xlarge"


model = JumpStartModel(
    model_id=model_id,
    name=endpoint_name,
)

start = time.time()

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    # predictor_cls=Predictor,
    endpoint_name=endpoint_name,
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=3600,
)

print(f"Elastic time: {time.time()-start}")  # model.deploy()

---------------------------------------------------------------------------------------------------------!Elastic time: 3498.1374440193176


### 4. Query endpoint and parse response

---
Input to the endpoint is any string of text formatted as json and encoded in `utf-8` format. Output of the endpoint is a `json` with generated text.

---

In [19]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"


def query_endpoint(encoded_text, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/x-text", Body=encoded_text
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions[0]["generated_text"]
    return generated_text

---
Below, we  put in some example input text. You can put in any text and the model predicts next words in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

---

In [24]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

text1 = "I will"
text2 = "The movie is so funny"
endpoint_name = "jumpstart-example-huggingface-textgener-2023-05-23-14-30-58-443"

for text in [text1, text2]:
    query_response = query_endpoint(json.dumps(text).encode("utf-8"), endpoint_name=endpoint_name)
    generated_text = parse_response(query_response)
    print(
        f"Inference:{newline}"
        f"input text: {text}{newline}"
        f"generated text: {bold}{generated_text}{unbold}{newline}"
    )

Inference:
input text: I will
generated text: ['"I will" is a future tense, so it is used to talk about what will happen in the']

Inference:
input text: The movie is so funny
generated text: ['"The movie is so funny"']



### 5. Advanced features: How to use various advanced parameters to control the generated text

***
This model also supports many advanced parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of stence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelyhood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

In [31]:
# Input must be a json
payload = {
    "text_inputs": ["How to make a pasta?"],
    "max_length": 2000,
    "num_return_sequences": 2,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
    "num_beams": 3,
}


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    # generated_text = model_predictions[0]["generated_text"]
    generated_text = []
    for x in model_predictions[0]:
        generated_text.append(x["generated_text"])
    return generated_text


query_response = query_endpoint_with_json_payload(
    json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
)

generated_texts = parse_response_multiple_texts(query_response)
print(generated_texts)

['How to make a pasta? boil water in a pot, then add the pasta.', 'How to make a pasta? boil water in a pot, add salt and pasta.']


### 6. Advanced features: How to use prompts engineering to solve different few shot and zero shot learning NLP tasks

<font color='red'>Note </font>.  The notebook in the following sections uses [**BloomZ 176B**](https://huggingface.co/bigscience/bloomz) as an example. You are welcome to try [**BloomZ 7b1**](https://huggingface.co/bigscience/bloomz-7b1), [**BloomZ 3b**](https://huggingface.co/bigscience/bloomz-3b) and many others yourself. Please modify `model_id` in above section.

## Let's first try few shot learning tasks

### 6.1. Summarization

Define the text article you want to summarize.

In [66]:
text = """Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition. """

In [67]:
prompts = [
    """[Original]: Amazon scientists, in collaboration with researchers from the University of Sheffield, are making a large-scale fact extraction and verification dataset publicly available for the first time. The dataset, comprising more than 185,000 evidence-backed claims, is being made available to hopefully catalyze research and development that addresses the problems of fact extraction and verification in software applications or cloud-based services that perform automatic information extraction.
    [Summary]: Amazon and University researchers make fact extraction and verification dataset publicly available.
    ###
    [Original]: Prime members in the U.S. can get even more delivered to their door with a Prime membership. Members can now enjoy one year of Grubhub+ valued at $9.99 per month for free—at no added cost to their Prime membership. To activate this deal, visit amazon.com/grubhub. This new offer includes unlimited, $0 food delivery fees on orders over $12 as well as exclusive perks for Grubhub+ members and rewards like free food and order discounts. Plus, diners can “eat good while doing good” by opting into Grubhub’s Donate the Change program, a donation-matching initiative that raised more than $25 million in 2021 alone, benefiting more than 20 charitable organizations across the country.
    [Summary]: Prime members in the U.S. can enjoy one year of Grubhub+ for free, with no food-delivery fees on eligible orders.
    ###
    [Original]: {text}
    [Summary]:"""
]

num_return_sequences = 2
parameters = {
    "max_length": 600,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.01,
    "do_sample": False,
}

print(f"{bold}Number of return sequences are set as {num_return_sequences}{unbold}{newline}")
for each_prompt in prompts:
    payload = {"text_inputs": each_prompt.replace("{text}", text), **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

Number of return sequences are set as 2

 For prompt: '[Original]: Amazon scientists, in collaboration with researchers from the University of Sheffield, are making a large-scale fact extraction and verification dataset publicly available for the first time. The dataset, comprising more than 185,000 evidence-backed claims, is being made available to hopefully catalyze research and development that addresses the problems of fact extraction and verification in software applications or cloud-based services that perform automatic information extraction.
    [Summary]: Amazon and University researchers make fact extraction and verification dataset publicly available.
    ###
    [Original]: Prime members in the U.S. can get even more delivered to their door with a Prime membership. Members can now enjoy one year of Grubhub+ valued at $9.99 per month for free—at no added cost to their Prime membership. To activate this deal, visit amazon.com/grubhub. This new offer includes unlimited, $0 foo

### 6.2. Code generation


In [68]:
description = "a Headline saying Welcome to AI"

In [69]:
prompts = [
    """description: a orange button that says stop
    code: <button style=color:white; background-color:orange;>Stop</button>
    ###
    description: a blue box that contains yellow circles with red borders
    code: <div style=background-color: blue; padding: 20px;><div style=background-color: yellow; border: 5px solid red; border-radius: 50%; padding: 20px; width: 100px; height: 100px;>
    ###
    description: {description}
    code:"""
]

num_return_sequences = 2

parameters = {
    "max_length": 200,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.3,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{description}", description)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'description: a orange button that says stop
    code: <button style=color:white; background-color:orange;>Stop</button>
    ###
    description: a blue box that contains yellow circles with red borders
    code: <div style=background-color: blue; padding: 20px;><div style=background-color: yellow; border: 5px solid red; border-radius: 50%; padding: 20px; width: 100px; height: 100px;>
    ###
    description: {description}
    code:'

 The 2 results are:

 The reasoning result 0 is: 'description: a orange button that says stop
    code: <button style=color:white; background-color:orange;>Stop</button>
    ###
    description: a blue box that contains yellow circles with red borders
    code: <div style=background-color: blue; padding: 20px;><div style=background-color: yellow; border: 5px solid red; border-radius: 50%; padding: 20px; width: 100px; height: 100px;>
    ###
    description: a Headline saying Welcome to AI
    code: <h1>Welcome to AI</h1>'

 The reasoning resu

How about SQL code?

In [70]:
question = "Fetch three employees from the Employee table"

In [71]:
prompts = [
    """Question: Fetch the companies that have less than five people in it.
    Answer: SELECT COMPANY, COUNT(EMPLOYEE_ID) FROM Employee GROUP BY COMPANY HAVING COUNT(EMPLOYEE_ID) < 5;
    ###
    Question: Show all companies along with the number of employees in each department
    Answer: SELECT COMPANY, COUNT(COMPANY) FROM Employee GROUP BY COMPANY;
    ###
    Question: Show the last record of the Employee table
    Answer: SELECT * FROM Employee ORDER BY LAST_NAME DESC LIMIT 1;
    ###
    Question: {question};
    Answer:"""
]

num_return_sequences = 2

parameters = {
    "max_length": 200,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.01,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'Question: Fetch the companies that have less than five people in it.
    Answer: SELECT COMPANY, COUNT(EMPLOYEE_ID) FROM Employee GROUP BY COMPANY HAVING COUNT(EMPLOYEE_ID) < 5;
    ###
    Question: Show all companies along with the number of employees in each department
    Answer: SELECT COMPANY, COUNT(COMPANY) FROM Employee GROUP BY COMPANY;
    ###
    Question: Show the last record of the Employee table
    Answer: SELECT * FROM Employee ORDER BY LAST_NAME DESC LIMIT 1;
    ###
    Question: {question};
    Answer:'

 The 2 results are:

 The reasoning result 0 is: 'Question: Fetch the companies that have less than five people in it.
    Answer: SELECT COMPANY, COUNT(EMPLOYEE_ID) FROM Employee GROUP BY COMPANY HAVING COUNT(EMPLOYEE_ID) < 5;
    ###
    Question: Show all companies along with the number of employees in each department
    Answer: SELECT COMPANY, COUNT(COMPANY) FROM Employee GROUP BY COMPANY;
    ###
    Question: Show the last record of the Employee 

### 6.3. Name entity recognition (NER)

In [72]:
sentence = "David Melvin is an investment and financial services professional at CITIC CLSA with over 30 years’ experience in investment banking and private equity. He is currently a Senior Adviser of CITIC CLSA."

In [73]:
prompts = [
    """[Text]: Fred is a serial entrepreneur. Co-founder and CEO of Platform.sh, he previously co-founded Commerce Guys, a leading Drupal ecommerce provider. His mission is to guarantee that as we continue on an ambitious journey to profoundly transform how cloud computing is used and perceived, we keep our feet well on the ground continuing the rapid growth we have enjoyed up until now. 
    [Name]: Fred
    [Position]: Co-founder and CEO
    [Company]: Platform.sh
    ###
    [Text]: Microsoft (the word being a portmanteau of "microcomputer software") was founded by Bill Gates on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800. Steve Ballmer replaced Gates as CEO in 2000, and later envisioned a "devices and services" strategy.
    [Name]:  Steve Ballmer
    [Position]: CEO
    [Company]: Microsoft
    ###
    [Text]: Franck Riboud was born on 7 November 1955 in Lyon. He is the son of Antoine Riboud, the previous CEO, who transformed the former European glassmaker BSN Group into a leading player in the food industry. He is the CEO at Danone.
    [Name]:  Franck Riboud
    [Position]: CEO
    [Company]: Danone
    ###
    [Text]: {sentence}
    """
]


parameters = {
    "max_length": 550,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: '[Text]: Fred is a serial entrepreneur. Co-founder and CEO of Platform.sh, he previously co-founded Commerce Guys, a leading Drupal ecommerce provider. His mission is to guarantee that as we continue on an ambitious journey to profoundly transform how cloud computing is used and perceived, we keep our feet well on the ground continuing the rapid growth we have enjoyed up until now. 
    [Name]: Fred
    [Position]: Co-founder and CEO
    [Company]: Platform.sh
    ###
    [Text]: Microsoft (the word being a portmanteau of "microcomputer software") was founded by Bill Gates on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800. Steve Ballmer replaced Gates as CEO in 2000, and later envisioned a "devices and services" strategy.
    [Name]:  Steve Ballmer
    [Position]: CEO
    [Company]: Microsoft
    ###
    [Text]: Franck Riboud was born on 7 November 1955 in Lyon. He is the son of Antoine Riboud, the previous CEO, who transformed the former Europea

### 6.4. Question answering

In [74]:
question = "Which plan is recommended for GPT-J?"

In [75]:
prompts = [
    """Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: When was NLP Cloud founded?
    Answer: 2021
    ###
    Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
    Question: What did NLP Cloud develop?
    Answer: API
    ###
    Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
    Question: When can plans be stopped?
    Answer: Anytime
    ###
    Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
    Question: {question}
    Answer:"""
]


parameters = {
    "max_length": 350,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: When was NLP Cloud founded?
    Answer: 2021
    ###
    Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
    Question: What did NLP Cloud develop?
    Answer: API
    ###
    Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
    Question: When can plans be stopped?
    Answer: Anytime
    ###
    Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
    Question: {question}
    Answer:'

 The 2 results are:

 The reasoning result 0 is: 'Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: Whe

### 6.5. Grammar and spelling correction 

In [76]:
sentence = "I do not wan to go"

In [77]:
prompts = [
    """I love goin to the beach.
    Correction: I love going to the beach.
    ###
    Let me hav it!
    Correction: Let me have it!
    ###
    It have too many drawbacks.
    Correction: It has too many drawbacks.
    ###
    {sentence}
    Correction:"""
]


parameters = {
    "max_length": 250,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'I love goin to the beach.
    Correction: I love going to the beach.
    ###
    Let me hav it!
    Correction: Let me have it!
    ###
    It have too many drawbacks.
    Correction: It has too many drawbacks.
    ###
    {sentence}
    Correction:'

 The 2 results are:

 The reasoning result 0 is: 'I love goin to the beach.
    Correction: I love going to the beach.
    ###
    Let me hav it!
    Correction: Let me have it!
    ###
    It have too many drawbacks.
    Correction: It has too many drawbacks.
    ###
    I do not wan to go
    Correction: I don't want to go
    ###'

 The reasoning result 1 is: 'I love goin to the beach.
    Correction: I love going to the beach.
    ###
    Let me hav it!
    Correction: Let me have it!
    ###
    It have too many drawbacks.
    Correction: It has too many drawbacks.
    ###
    I do not wan to go
    Correction: I don't want to go
    ###'



### 6.6. Product description and generalization


In [78]:
sentence = """t-shirt, men, $39"""

In [79]:
prompts = [
    """Generate a product description out of keywords.

    Keywords: shoes, women, $59
    Sentence: Beautiful shoes for women at the price of $59.
    ###
    Keywords: trousers, men, $69
    Sentence: Modern trousers for men, for $69 only.
    ###
    Keywords: gloves, winter, $19
    Sentence: Amazingly hot gloves for cold winters, at $19.
    ###
    Keywords: {sentence}
    Sentence:"""
]


parameters = {
    "max_length": 150,
    "num_return_sequences": num_return_sequences,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'Generate a product description out of keywords.

    Keywords: shoes, women, $59
    Sentence: Beautiful shoes for women at the price of $59.
    ###
    Keywords: trousers, men, $69
    Sentence: Modern trousers for men, for $69 only.
    ###
    Keywords: gloves, winter, $19
    Sentence: Amazingly hot gloves for cold winters, at $19.
    ###
    Keywords: {sentence}
    Sentence:'

 The 2 results are:

 The reasoning result 0 is: 'Generate a product description out of keywords.

    Keywords: shoes, women, $59
    Sentence: Beautiful shoes for women at the price of $59.
    ###
    Keywords: trousers, men, $69
    Sentence: Modern trousers for men, for $69 only.
    ###
    Keywords: gloves, winter, $19
    Sentence: Amazingly hot gloves for cold winters, at $19.
    ###
    Keywords: t-shirt, men, $39
    Sentence: Cool t-shirt for men, at $39.'

 The reasoning result 1 is: 'Generate a product description out of keywords.

    Keywords: shoes, women, $59
    Sentence:

### 6.7. Sentence / Sentiment Classification

Define the sentence you want to classifiy and the corresponded options.

In [80]:
sentence = "I am trying to cook chicken with tomatoes."

In [81]:
prompts = [
    """Message: When the spaceship landed on Mars, the whole humanity was excited
    Topic: space
    ###
    Message: I love playing tennis and golf. I'm practicing twice a week.
    Topic: sport
    ###
    Message: Managing a team of sales people is a tough but rewarding job.
    Topic: business
    ###
    Message: {sentence}
    Topic:"""
]

parameters = {
    "max_length": 200,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'Message: When the spaceship landed on Mars, the whole humanity was excited
    Topic: space
    ###
    Message: I love playing tennis and golf. I'm practicing twice a week.
    Topic: sport
    ###
    Message: Managing a team of sales people is a tough but rewarding job.
    Topic: business
    ###
    Message: I am trying to cook chicken with tomatoes.
    Topic:'

 The 2 results are:

 The reasoning result 0 is: 'Message: When the spaceship landed on Mars, the whole humanity was excited
    Topic: space
    ###
    Message: I love playing tennis and golf. I'm practicing twice a week.
    Topic: sport
    ###
    Message: Managing a team of sales people is a tough but rewarding job.
    Topic: business
    ###
    Message: I am trying to cook chicken with tomatoes.
    Topic: food'

 The reasoning result 1 is: 'Message: When the spaceship landed on Mars, the whole humanity was excited
    Topic: space
    ###
    Message: I love playing tennis and golf. I'm practicing 

### 6.8. Chatbot and Conversational AI

Define the sentence and the language you want to translate the sentence to.

In [82]:
sentence = "I caught flu"

In [83]:
prompts = [
    """This is a discussion between a [human] and a [robot]. 
    The [robot] is very nice and empathetic.

    [human]: Hello nice to meet you.
    [robot]: Nice to meet you too.
    ###
    [human]: How is it going today?
    [robot]: Not so bad, thank you! How about you?
    ###
    [human]: I am ok, but I am a bit sad...
    [robot]: Oh? Why that?
    ###
    [human]: {sentence}
    [robot]:"""
]

parameters = {
    "max_length": 250,
    "num_return_sequences": num_return_sequences,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'This is a discussion between a [human] and a [robot]. 
    The [robot] is very nice and empathetic.

    [human]: Hello nice to meet you.
    [robot]: Nice to meet you too.
    ###
    [human]: How is it going today?
    [robot]: Not so bad, thank you! How about you?
    ###
    [human]: I am ok, but I am a bit sad...
    [robot]: Oh? Why that?
    ###
    [human]: I caught flu
    [robot]:'

 The 2 results are:

 The reasoning result 0 is: 'This is a discussion between a [human] and a [robot]. 
    The [robot] is very nice and empathetic.

    [human]: Hello nice to meet you.
    [robot]: Nice to meet you too.
    ###
    [human]: How is it going today?
    [robot]: Not so bad, thank you! How about you?
    ###
    [human]: I am ok, but I am a bit sad...
    [robot]: Oh? Why that?
    ###
    [human]: I caught flu
    [robot]: Oh no, I am sorry to hear that.'

 The reasoning result 1 is: 'This is a discussion between a [human] and a [robot]. 
    The [robot] is very nice

### 6.9. Tweet generation

Define the sentence, pronoun, and options you want to reason.

In [84]:
keyword = "nlp"

In [85]:
prompts = [
    """keyword: markets
    tweet: Take feedback from nature and markets, not from people
    ###
    keyword: children
    tweet: Maybe we die so we can come back as children.
    ###
    keyword: startups
    tweet: Startups should not worry about how to put out fires, they should worry about how to start them.
    ###
    keyword: {keyword}
    tweet:"""
]

parameters = {
    "max_length": 300,
    "num_return_sequences": num_return_sequences,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{keyword}", keyword)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'keyword: markets
    tweet: Take feedback from nature and markets, not from people
    ###
    keyword: children
    tweet: Maybe we die so we can come back as children.
    ###
    keyword: startups
    tweet: Startups should not worry about how to put out fires, they should worry about how to start them.
    ###
    keyword: nlp
    tweet:'

 The 2 results are:

 The reasoning result 0 is: 'keyword: markets
    tweet: Take feedback from nature and markets, not from people
    ###
    keyword: children
    tweet: Maybe we die so we can come back as children.
    ###
    keyword: startups
    tweet: Startups should not worry about how to put out fires, they should worry about how to start them.
    ###
    keyword: nlp
    tweet: NLP is a tool, not a goal.'

 The reasoning result 1 is: 'keyword: markets
    tweet: Take feedback from nature and markets, not from people
    ###
    keyword: children
    tweet: Maybe we die so we can come back as children.
    ###
    keywor

### 6.10. Machine translation

In [86]:
sentence = "NLP Cloud permet de deployer le NLP en production facilement."

In [87]:
prompts = [
    """Hugging Face a révolutionné le NLP.
    Translation: Hugging Face revolutionized NLP.
    ###
    Cela est incroyable!
    Translation: This is unbelievable!
    ###
    Désolé je ne peux pas.
    Translation: Sorry but I cannot.
    ###
    {sentence}
    Translation:"""
]

parameters = {
    "max_length": 150,
    "num_return_sequences": num_return_sequences,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'Hugging Face a révolutionné le NLP.
    Translation: Hugging Face revolutionized NLP.
    ###
    Cela est incroyable!
    Translation: This is unbelievable!
    ###
    Désolé je ne peux pas.
    Translation: Sorry but I cannot.
    ###
    NLP Cloud permet de deployer le NLP en production facilement.
    Translation:'

 The 2 summarized results are:

 The reasoning result 0 is: 'Hugging Face a révolutionné le NLP.
    Translation: Hugging Face revolutionized NLP.
    ###
    Cela est incroyable!
    Translation: This is unbelievable!
    ###
    Désolé je ne peux pas.
    Translation: Sorry but I cannot.
    ###
    NLP Cloud permet de deployer le NLP en production facilement.
    Translation: NLP Cloud makes it easy to deploy NLP in production.'

 The reasoning result 1 is: 'Hugging Face a révolutionné le NLP.
    Translation: Hugging Face revolutionized NLP.
    ###
    Cela est incroyable!
    Translation: This is unbelievable!
    ###
    Désolé je ne peux pas.
    

### 6.11. Intent classification

In [88]:
sentence = "Can you please teach me Chinese next week?"

In [89]:
prompts = [
    """I want to start coding tomorrow because it seems to be so fun!
    Intent: start coding
    ###
    Show me the last pictures you have please.
    Intent: show pictures
    ###
    Search all these files as fast as possible.
    Intent: search files
    ###
    {sentence}
    Intent:"""
]


parameters = {
    "max_length": 150,
    "num_return_sequences": num_return_sequences,
    "top_p": 0.5,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: 'I want to start coding tomorrow because it seems to be so fun!
    Intent: start coding
    ###
    Show me the last pictures you have please.
    Intent: show pictures
    ###
    Search all these files as fast as possible.
    Intent: search files
    ###
    Can you please teach me Chinese next week?
    Intent:'

 The 2 summarized results are:

 The reasoning result 0 is: 'I want to start coding tomorrow because it seems to be so fun!
    Intent: start coding
    ###
    Show me the last pictures you have please.
    Intent: show pictures
    ###
    Search all these files as fast as possible.
    Intent: search files
    ###
    Can you please teach me Chinese next week?
    Intent: teach me chinese
    ###'

 The reasoning result 1 is: 'I want to start coding tomorrow because it seems to be so fun!
    Intent: start coding
    ###
    Show me the last pictures you have please.
    Intent: show pictures
    ###
    Search all these files as fast as possible.
    Inte

## Next, let's try zero shot learning tasks

### 6.13. Multilingual text / sentiment classification (Chinese to English)

In [91]:
text = """一个传奇的开端，一个不灭的神话，这不仅仅是一部电影，而是作为一个走进新时代的标签，永远彪炳史册。"""

In [94]:
prompts = ["""{text} Would you rate the previous review as positive, neutral or negative?"""]

parameters = {
    "max_length": 200,
    "num_return_sequences": num_return_sequences,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
}

for each_prompt in prompts:
    input_text = each_prompt.replace("{text}", text)
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"For input with prompt: {payload['text_inputs']}")
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

For input with prompt: 一个传奇的开端，一个不灭的神话，这不仅仅是一部电影，而是作为一个走进新时代的标签，永远彪炳史册。 Would you rate the previous review as positive, neutral or negative?
 The 2 results are:

 The reasoning result 0 is: '一个传奇的开端，一个不灭的神话，这不仅仅是一部电影，而是作为一个走进新时代的标签，永远彪炳史册。 Would you rate the previous review as positive, neutral or negative? positive'

 The reasoning result 1 is: '一个传奇的开端，一个不灭的神话，这不仅仅是一部电影，而是作为一个走进新时代的标签，永远彪炳史册。 Would you rate the previous review as positive, neutral or negative? positive'



### 6.14. Multilingual question and answering (English to Chinese)

In [95]:
text = """what is the backpropagation."""

In [99]:
prompts = ["""Explain to me in Traditional Chinese  {text}"""]

parameters = {
    "max_length": 500,
    "num_return_sequences": num_return_sequences,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
}

for each_prompt in prompts:
    input_text = each_prompt.replace("{text}", text)
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"For input with prompt: {payload['text_inputs']}")
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

For input with prompt: Explain to me in Traditional Chinese  what is the backpropagation.
 The 2 results are:

 The reasoning result 0 is: 'Explain to me in Traditional Chinese  what is the backpropagation. 梯度反向传播算法是一种机器学习算法，用于训练人工神经网络。该算法使用梯度下降来最小化损失函数。'

 The reasoning result 1 is: 'Explain to me in Traditional Chinese  what is the backpropagation. 遞迴神經網絡的反向傳播算法，是一種用於訓練遞迴神經網絡的算法。'



### 6.15. Code generation

In [100]:
text = "binary search tree"
code_start = """
def binary_search(a, x):
    low = 0
    high = len(a) - 1"""

In [101]:
prompts = [
    """Write a {text} with O(log(n)) computational complexity. 
{code_start}"""
]

In [102]:
parameters = {
    "max_length": 500,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": False,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{text}", text)
    input_text = input_text.replace("{code_start}", code_start)
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"For input with prompt: {payload['text_inputs']}")
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

For input with prompt: Write a binary search tree with O(log(n)) computational complexity. 

def binary_search(a, x):
    low = 0
    high = len(a) - 1
 For prompt: 'Write a {text} with O(log(n)) computational complexity. 
{code_start}'

 The 2 results are:

 The reasoning result 0 is: 'Write a binary search tree with O(log(n)) computational complexity. 

def binary_search(a, x):
    low = 0
    high = len(a) - 1
    while low <= high:
        mid = (low + high) // 2
        if a[mid] < x:
            low = mid + 1
        elif a[mid] > x:
            high = mid - 1
        else:
            return mid
    return -1'

 The reasoning result 1 is: 'Write a binary search tree with O(log(n)) computational complexity. 

def binary_search(a, x):
    low = 0
    high = len(a) - 1
    while low <= high:
        mid = (low + high) // 2
        if a[mid] < x:
            low = mid + 1
        elif a[mid] > x:
            high = mid - 1
        else:
            return mid
    return low'



### 6.16. Paragraph rephrase

In [103]:
sentence = """Amazon Web Services (AWS) has announced nine major new updates for its cloud-based machine learning platform, SageMaker.
SageMaker aims to provide a machine learning service which can be used to build, train, and deploy ML models for virtually any use case.
During this year’s re:Invent conference, AWS made several announcements to further improve SageMaker’s capabilities."""

In [105]:
prompts = [
    """\ {sentence}\n\nHow would you rephrase that briefly using English?""",
    """"{sentence}\nThe above sentence is very complicated. Please provide me a simplified synonymous version consisting of multiple sentences:""",
]

parameters = {
    "max_length": 5000,
    "num_return_sequences": num_return_sequences,
    "top_k": 250,
    "top_p": 0.01,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold} The reasoning result {idx} is{unbold}: '{each_generated_text}'{newline}")

 For prompt: '\ Amazon Web Services (AWS) has announced nine major new updates for its cloud-based machine learning platform, SageMaker.
SageMaker aims to provide a machine learning service which can be used to build, train, and deploy ML models for virtually any use case.
During this year’s re:Invent conference, AWS made several announcements to further improve SageMaker’s capabilities.

How would you rephrase that briefly using English?'

 The 2 results are:

 The reasoning result 0 is: '\ Amazon Web Services (AWS) has announced nine major new updates for its cloud-based machine learning platform, SageMaker.
SageMaker aims to provide a machine learning service which can be used to build, train, and deploy ML models for virtually any use case.
During this year’s re:Invent conference, AWS made several announcements to further improve SageMaker’s capabilities.

How would you rephrase that briefly using English? Amazon SageMaker gets a major update'

 The reasoning result 1 is: '\ Amazon

### 6.17. Summarization

In [106]:
text = """Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition. 
All of the Amazon Comprehend features accept UTF-8 text documents as the input. In addition, custom classification and custom entity recognition accept image files, PDF files, and Word files as input. 
Amazon Comprehend can examine and analyze documents in a variety of languages, depending on the specific feature. For more information, see Languages supported in Amazon Comprehend. Amazon Comprehend's Dominant language capability can examine documents and determine the dominant language for a far wider selection of languages."""

In [107]:
prompts = ["""{text}\n\n===\nWrite a summary of the previous text in English:"""]

num_return_sequences = 2
parameters = {
    "max_length": 500,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}

print(f"{bold}Number of return sequences are set as {num_return_sequences}{unbold}{newline}")
for each_prompt in prompts:
    payload = {"text_inputs": each_prompt.replace("{text}", text), **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} summarized results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

Number of return sequences are set as 2

 For prompt: '{text}

===
Write a summary of the previous text in English:'

 The 2 summarized results are:

Result 0: Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own c

### 6.18. Common sense reasoning / natural language inference

In the common sense reasoning, you can design a prompt and combine it with the premise, hypothesis, and options, send the combined text into the endpoint to get an answer. Examples are demonstrated as below.

In [108]:
premise = "The world cup has kicked off in Los Angeles, United States."
hypothesis = "The world cup takes place in United States."
options = """["yes", "no"]"""

In [111]:
prompts = [
    """Given that {premise} Does it follow that {hypothesis} Yes or no?""",
    """"{premise} Using only the above description and what you know about the world, is \"{hypothesis}\" definitely correct? Yes or no?""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{premise}", premise)
    input_text = input_text.replace("{hypothesis}", hypothesis)
    # input_text = input_text.replace("{options_}", options)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

 For prompt: 'Given that The world cup has kicked off in Los Angeles, United States. Does it follow that The world cup takes place in United States. Yes or no?'

 The 2 results are:

Result 0: Given that The world cup has kicked off in Los Angeles, United States. Does it follow that The world cup takes place in United States. Yes or no? Yes

Result 1: Given that The world cup has kicked off in Los Angeles, United States. Does it follow that The world cup takes place in United States. Yes or no? Yes

 For prompt: '"The world cup has kicked off in Los Angeles, United States. Using only the above description and what you know about the world, is "The world cup takes place in United States." definitely correct? Yes or no?'

 The 2 results are:

Result 0: "The world cup has kicked off in Los Angeles, United States. Using only the above description and what you know about the world, is "The world cup takes place in United States." definitely correct? Yes or no? Yes

Result 1: "The world cup 

### 6.19. Question and Answering

Now, let's try another reasoning task with a different type of prompt template. You can simply provide context and question as shown below.

In [112]:
context = """The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would on paper. They also attach magnetically to your Kindle and never need to be charged. The Premium Pen includes a dedicated eraser and a customizable shortcut button.

The Kindle Scribe has the most storage options of all Kindle devices: choose from 8 GB, 16 GB, or 32 GB to suit your level of reading and writing.
"""
question = "what are the key features of new Kindle?"

In [114]:
prompts = ["""question: \"{question}"\\n\nContext: \"{context}"\\n\nAnswer:"""]


parameters = {
    "max_length": 500,
    "num_return_sequences": num_return_sequences,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{context}", context)
    input_text = input_text.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

 For prompt: 'question: "{question}"\n
Context: "{context}"\n
Answer:'

 The 2 results are:

Result 0: question: "what are the key features of new Kindle?"\n
Context: "The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light,

### 6.20. Sentence / Sentiment Classification

In [115]:
sentence1 = "This moive is so great and once again dazzles and delights us"
options_ = """OPTIONS:\n-positive \n-negative """

In [116]:
prompts = [
    """Review:\n{sentence}\nIs this movie review sentence negative or positive?\n{options_}""",
    """Short movie review: {sentence}\nDid the critic think positively or negatively of the movie?\n{options_}""",
    """Sentence from a movie review: {sentence}\nWas the movie seen positively or negatively based on the preceding review? \n\n{options_}""",
    """\"{sentence}\"\nHow would the sentiment of this sentence be perceived?\n\n{options_}""",
    """Is the sentiment of the following sentence positive or negative?\n{sentence}\n{options_}""",
    """What is the sentiment of the following movie review sentence?\n{sentence}\n{options_}""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence1)
    input_text = input_text.replace("{options_}", options_)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

 For prompt: 'Review:
This moive is so great and once again dazzles and delights us
Is this movie review sentence negative or positive?
OPTIONS:
-positive 
-negative '

 The 2 results are:

Result 0: Review:
This moive is so great and once again dazzles and delights us
Is this movie review sentence negative or positive?
OPTIONS:
-positive 
-negative  Positive

Result 1: Review:
This moive is so great and once again dazzles and delights us
Is this movie review sentence negative or positive?
OPTIONS:
-positive 
-negative  Positive

 For prompt: 'Short movie review: This moive is so great and once again dazzles and delights us
Did the critic think positively or negatively of the movie?
OPTIONS:
-positive 
-negative '

 The 2 results are:

Result 0: Short movie review: This moive is so great and once again dazzles and delights us
Did the critic think positively or negatively of the movie?
OPTIONS:
-positive 
-negative  positive

Result 1: Short movie review: This moive is so great and once

### 6.21. Imaginary article generation based on title

In [120]:
title = "University has new facility coming up"

In [121]:
prompts = [
    """Title: \"{title}\"\\nGiven the above title of an imaginary article, imagine the article.\\n"""
]


parameters = {
    "max_length": 5000,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{title}", title)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

 For prompt: 'Title: "University has new facility coming up"\nGiven the above title of an imaginary article, imagine the article.\n'

 The 2 results are:

Result 0: Title: "University has new facility coming up"\nGiven the above title of an imaginary article, imagine the article.\n The new £5.7m facility will be built on the site of the existing £3.2m building at the University of the Highlands and Islands (UHI) campus in Inverness.

It will include a new lecture theatre, seminar room, office space and a new entrance.

UHI said the new building would be completed by the end of the year.

The existing building was completed in 2004 and has been used for teaching and research since then.

UHI said the new building would allow it to expand its provision of higher education in the Highlands and Islands.

It said the new building would also allow it to increase the number of students it could accommodate.

UHI said it had made a number of changes to its provision of higher education in the 

### 6.22. Summarize a title based on the article

In [122]:
article = """The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would on paper. They also attach magnetically to your Kindle and never need to be charged. The Premium Pen includes a dedicated eraser and a customizable shortcut button.

The Kindle Scribe has the most storage options of all Kindle devices: choose from 8 GB, 16 GB, or 32 GB to suit your level of reading and writing."""

In [124]:
prompts = ["""'\'{article} \n\n \\n\\nGive me a good title for the article above."""]

parameters = {
    "max_length": 5000,
    "num_return_sequences": num_return_sequences,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{article}", article)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The {num_return_sequences} results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

 For prompt: '''The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would

### 7. Clean up the endpoint

In [90]:
# Delete the SageMaker endpoint
model_predictor.delete_model()
model_predictor.delete_endpoint()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|text-generation-few-shot-learning.ipynb)
